<a href="https://colab.research.google.com/github/UEC-MaTa/News_Aggregator/blob/main/%E7%B5%90%E8%AB%96%EF%BC%89%E4%B8%80%E3%81%8B%E3%82%89%E3%83%87%E3%82%A3%E3%83%AC%E3%82%AF%E3%83%88%E3%83%AA%E3%81%AB%E5%88%86%E3%81%91%E3%81%9F%E3%83%87%E3%83%BC%E3%82%BF%E3%82%BB%E3%83%83%E3%83%88%E3%82%92%E4%BD%9C%E3%82%8B%E3%81%AE%E3%81%AF%E3%83%8A%E3%83%B3%E3%82%BB%E3%83%B3%E3%82%B9%E3%81%8B%EF%BC%9F%E3%80%80%E3%83%8B%E3%83%A5%E3%83%BC%E3%82%B9%E3%83%87%E3%83%BC%E3%82%BF%E3%82%92%E3%82%AB%E3%83%86%E3%82%B4%E3%83%AA%E3%83%BC%E3%81%94%E3%81%A8%E3%81%AB%E5%88%86%E9%A1%9E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses

#importing libraries
import pandas as pd
import numpy as np
import seaborn as sns
import re
from tensorflow import keras
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout
from tensorflow.keras.models import Sequential
from keras.utils.np_utils import to_categorical

In [1]:
# 下記セルを実行すると、authorization codeの入力を求められます。
# 出力されたリンク先をクリックし、Googleアカウントにログインし、
# authorization codeをコピーし、貼り付けをおこなってください。
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import pandas as pd

#newsCorpora.csv
newsCorpora = pd.read_csv('/content/drive/MyDrive/Tensorflow/NewsAggregator/newsCorpora.csv',sep='\t',  names=['ID', 'TITLE', 'URL', 'PUBLISHER', 'CATEGORY', 'STORY', 'HOSTNAME', 'TIMESTAMP'])
display(newsCorpora.shape)
display(newsCorpora)

(422419, 8)

,ID,TITLE,URL,PUBLISHER,CATEGORY,STORY,HOSTNAME,TIMESTAMP
0,1,"Fed official says weak data caused by weather,...",http://www.latimes.com/business/money/la-fi-mo...,Los Angeles Times,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.latimes.com,1394470370698
1,2,Fed's Charles Plosser sees high bar for change...,http://www.livemint.com/Politics/H2EvwJSK2VE6O...,Livemint,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.livemint.com,1394470371207
2,3,US open: Stocks fall after Fed official hints ...,http://www.ifamagazine.com/news/us-open-stocks...,IFA Magazine,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.ifamagazine.com,1394470371550
3,4,"Fed risks falling 'behind the curve', Charles ...",http://www.ifamagazine.com/news/fed-risks-fall...,IFA Magazine,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.ifamagazine.com,1394470371793
4,5,Fed's Plosser: Nasty Weather Has Curbed Job Gr...,http://www.moneynews.com/Economy/federal-reser...,Moneynews,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.moneynews.com,1394470372027
...,...,...,...,...,...,...,...,...
422414,422933,Surgeons to remove 4-year-old's rib to rebuild...,http://www.cbs3springfield.com/story/26378648/...,WSHM-TV,m,dpcLMoJD69UYMXMxaoEFnWql9YjQM,www.cbs3springfield.com,1409229190251
422415,422934,Boy to have surgery on esophagus after battery...,http://www.wlwt.com/news/boy-to-have-surgery-o...,WLWT Cincinnati,m,dpcLMoJD69UYMXMxaoEFnWql9YjQM,www.wlwt.com,1409229190508
422416,422935,Child who swallowed battery to have reconstruc...,http://www.newsnet5.com/news/local-news/child-...,NewsNet5.com,m,dpcLMoJD69UYMXMxaoEFnWql9YjQM,www.newsnet5.com,1409229190771
422417,422936,Phoenix boy undergoes surgery to repair throat...,http://www.wfsb.com/story/26368078/phoenix-boy...,WFSB,m,dpcLMoJD69UYMXMxaoEFnWql9YjQM,www.wfsb.com,1409229191071


In [6]:

#ファイル格納するディレクトリを作る
import os
import shutil

_all_news_path = os.path.join('/content/drive/MyDrive/Tensorflow/NewsAggregator', 'all_news')
os.mkdir(_all_news_path)

_train_news_path = os.path.join('/content/drive/MyDrive/Tensorflow/NewsAggregator', 'all_news','train')
_test_news_path = os.path.join('/content/drive/MyDrive/Tensorflow/NewsAggregator', 'all_news','test')
os.mkdir(_train_news_path)
os.mkdir(_test_news_path)

In [7]:
#creating a new dataset with only relevant features.
ds = newsCorpora[['TITLE','CATEGORY']]

print(ds.head()) #★

#shuffling rows with the help of sample, here (frac = 1) means return all rows
ds = ds.sample(frac=1).reset_index(drop=True)

print(ds.head())

                                               TITLE CATEGORY
0  Fed official says weak data caused by weather,...        b
1  Fed's Charles Plosser sees high bar for change...        b
2  US open: Stocks fall after Fed official hints ...        b
3  Fed risks falling 'behind the curve', Charles ...        b
4  Fed's Plosser: Nasty Weather Has Curbed Job Gr...        b
                                               TITLE CATEGORY
0     AbbVie finalizes deal to buy Lexington's Shire        b
1  ABC Pulls Promos Over Molestation Accusations ...        e
2  Microsoft Promises to Stop Snooping Through Yo...        t
3             Transcendence Interview: Wally Pfister        e
4  Android Wear experiencing glitches with paid apps        t


In [ ]:
#splitting the Dataset into Train and Test set
x_train,x_test,y_train,y_test=train_test_split(ds['TITLE'],ds['CATEGORY'],test_size=0.2,random_state=42)
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(337935,) (337935,)
(84484,) (84484,)


In [ ]:


_train_images_dir_All_path = os.path.join('/','content', 'drive','MyDrive','SIGNATE','Images_labeling', 'train_dir_All', 'train')
os.mkdir(_train_images_dir_All_path)  # 100画像に限定
os.chdir(_train_images_dir_All_path)  # 絶対パス

for dir in range(len(label_master)):
  label_name = label_master["label_name"][dir] # label_nameの列から個々の要素を取り出す
  os.mkdir(label_name)  # サンプルディレクトリの作成

#ret = shutil.copy('a.txt', 'aquatic_mammals')  # dstにディレクトリを指定する
#print(ret)  # foo/a.txt：指定したディレクトリの下にファイルをコピー

FileExistsError: ignored

In [ ]:
#検証サンプルディレクトリの作成
#すべてのファイルを対象にするには"train_dir"、_All個に限定するには"train_dir_All"

import shutil

_train_images_valdir_All_path = os.path.join('/','content', 'drive','MyDrive','SIGNATE','Images_labeling', 'train_dir_All', 'val')
os.mkdir(_train_images_valdir_All_path)  # _All画像に限定
os.chdir(_train_images_valdir_All_path)  # 絶対パス

for dir in range(len(label_master)):
  label_name = label_master["label_name"][dir] # label_nameの列から個々の要素を取り出す
  os.mkdir(label_name)  # サンプルディレクトリの作成

#ret = shutil.copy('a.txt', 'aquatic_mammals')  # dstにディレクトリを指定する
#print(ret)  # foo/a.txt：指定したディレクトリの下にファイルをコピー

In [ ]:
"""
for dir in range(len(label_master)):
  label_name = label_master["label_name"][dir] # label_nameの列から個々の要素を取り出す
  print(label_name)
"""

In [ ]:
train_master_trial = train_master[:] #+++++++
display(train_master_trial)

,file_name,label_id
0,train_00000.png,11
1,train_00001.png,15
2,train_00002.png,4
3,train_00003.png,14
4,train_00004.png,1
...,...,...
49995,train_49995.png,16
49996,train_49996.png,7
49997,train_49997.png,8
49998,train_49998.png,7


In [ ]:
import numpy as np

display(test_master)

val_master_trial = pd.DataFrame(test_master) #+++++++ test_masterをコピー　label_idの取り扱いについては要検討
val_master_trial = val_master_trial.rename(columns={val_master_trial.columns[0]: 'file_name'})
display(val_master_trial)

0       test_00000.png
1       test_00001.png
2       test_00002.png
3       test_00003.png
4       test_00004.png
             ...      
9995    test_09995.png
9996    test_09996.png
9997    test_09997.png
9998    test_09998.png
9999    test_09999.png
Name: 0, Length: 10000, dtype: object

,file_name
0,test_00000.png
1,test_00001.png
2,test_00002.png
3,test_00003.png
4,test_00004.png
...,...
9995,test_09995.png
9996,test_09996.png
9997,test_09997.png
9998,test_09998.png


In [ ]:
import shutil

for file_n in range(len(train_master_trial)):
  f_name = train_master_trial["file_name"][file_n]
  l_id = train_master_trial["label_id"][file_n]
  l_name = label_master["label_name"][l_id]
  print(file_n, f_name, l_id, l_name)
  _dir_path = os.path.join('/','content', 'drive','MyDrive','SIGNATE','Images_labeling', 'train_dir_All','train', l_name)
  ret = shutil.copy('/content/drive/MyDrive/SIGNATE/Images_labeling/train/'+f_name, _dir_path)  # dstにディレクトリを指定する
print(ret)  # foo/a.txt：指定したディレクトリの下にファイルをコピー

ストリーミング出力は最後の 5000 行に切り捨てられました。
44001 train_44001.png 17 trees
44002 train_44002.png 16 small_mammals
44003 train_44003.png 6 household_furniture
44004 train_44004.png 13 non-insect_invertebrates
44005 train_44005.png 13 non-insect_invertebrates
44006 train_44006.png 11 large_omnivores_and_herbivores
44007 train_44007.png 0 aquatic_mammals
44008 train_44008.png 2 flowers
44009 train_44009.png 1 fish
44010 train_44010.png 11 large_omnivores_and_herbivores
44011 train_44011.png 16 small_mammals
44012 train_44012.png 8 large_carnivores
44013 train_44013.png 16 small_mammals
44014 train_44014.png 19 vehicles_2
44015 train_44015.png 18 vehicles_1
44016 train_44016.png 6 household_furniture
44017 train_44017.png 12 medium_mammals
44018 train_44018.png 11 large_omnivores_and_herbivores
44019 train_44019.png 9 large_man-made_outdoor_things
44020 train_44020.png 12 medium_mammals
44021 train_44021.png 9 large_man-made_outdoor_things
44022 train_44022.png 3 food_containers
44023 train_44023.png 

In [ ]:
import shutil

for file_n in range(len(val_master_trial)):  #testデータを利用
  v_f_name = val_master_trial["file_name"][file_n]
  #v_l_id = val_master_trial["label_id"][file_n]
  #l_name = label_master["label_name"][v_l_id]
  #print(file_n, v_f_name, v_l_id, l_name)
  l_name = "no_label"
  print(file_n, v_f_name, l_name)
  _valdir_path = os.path.join('/','content', 'drive','MyDrive','SIGNATE','Images_labeling', 'train_dir_All','val')
  ret = shutil.copy('/content/drive/MyDrive/SIGNATE/Images_labeling/test/'+v_f_name, _valdir_path)  # dstにディレクトリを指定する
print(ret)  # foo/a.txt：指定したディレクトリの下にファイルをコピー

ストリーミング出力は最後の 5000 行に切り捨てられました。
5001 test_05001.png no_label
5002 test_05002.png no_label
5003 test_05003.png no_label
5004 test_05004.png no_label
5005 test_05005.png no_label
5006 test_05006.png no_label
5007 test_05007.png no_label
5008 test_05008.png no_label
5009 test_05009.png no_label
5010 test_05010.png no_label
5011 test_05011.png no_label
5012 test_05012.png no_label
5013 test_05013.png no_label
5014 test_05014.png no_label
5015 test_05015.png no_label
5016 test_05016.png no_label
5017 test_05017.png no_label
5018 test_05018.png no_label
5019 test_05019.png no_label
5020 test_05020.png no_label
5021 test_05021.png no_label
5022 test_05022.png no_label
5023 test_05023.png no_label
5024 test_05024.png no_label
5025 test_05025.png no_label
5026 test_05026.png no_label
5027 test_05027.png no_label
5028 test_05028.png no_label
5029 test_05029.png no_label
5030 test_05030.png no_label
5031 test_05031.png no_label
5032 test_05032.png no_label
5033 test_05033.png no_label
5034 test_0

In [ ]:
import shutil

_news_dir_path = os.path.join('/content/drive/MyDrive/Tensorflow/NewsAggregator', 'news_dir')
os.mkdir(_news_dir_path)

for category in ["b", "t", "e", "m"]: #b = business, t = science and technology, e = entertainment, m = health
  _category_dir_path = os.path.join('/content/drive/MyDrive/Tensorflow/NewsAggregator', 'news_dir', category)
  os.mkdir(_category_dir_path)  # サンプルディレクトリの作成

#ret = shutil.copy('a.txt', 'aquatic_mammals')  # dstにディレクトリを指定する
#print(ret)  # foo/a.txt：指定したディレクトリの下にファイルをコピー

In [ ]:
import shutil

for n in range(len(newsCorpora)):
  id = newsCorpora[0][n]
  category = newsCorpora[4][n]
#  print(n, category, id)
  _dir_path = os.path.join('/content/drive/MyDrive/Tensorflow/NewsAggregator/news_dir', category)
  ret = shutil.copy(str(id), _dir_path)  # dstにディレクトリを指定する
#print(ret)  # foo/a.txt：指定したディレクトリの下にファイルをコピー

FileNotFoundError: ignored

In [ ]:
import shutil

for file_n in range(len(train_master_trial)):
  f_name = train_master_trial["file_name"][file_n]
  l_id = train_master_trial["label_id"][file_n]
  l_name = label_master["label_name"][l_id]
  print(file_n, f_name, l_id, l_name)
  _dir_path = os.path.join('/','content', 'drive','MyDrive','SIGNATE','Images_labeling', 'train_dir_All','train', l_name)
  ret = shutil.copy('/content/drive/MyDrive/SIGNATE/Images_labeling/train/'+f_name, _dir_path)  # dstにディレクトリを指定する
print(ret)  # foo/a.txt：指定したディレクトリの下にファイルをコピー

ストリーミング出力は最後の 5000 行に切り捨てられました。
44001 train_44001.png 17 trees
44002 train_44002.png 16 small_mammals
44003 train_44003.png 6 household_furniture
44004 train_44004.png 13 non-insect_invertebrates
44005 train_44005.png 13 non-insect_invertebrates
44006 train_44006.png 11 large_omnivores_and_herbivores
44007 train_44007.png 0 aquatic_mammals
44008 train_44008.png 2 flowers
44009 train_44009.png 1 fish
44010 train_44010.png 11 large_omnivores_and_herbivores
44011 train_44011.png 16 small_mammals
44012 train_44012.png 8 large_carnivores
44013 train_44013.png 16 small_mammals
44014 train_44014.png 19 vehicles_2
44015 train_44015.png 18 vehicles_1
44016 train_44016.png 6 household_furniture
44017 train_44017.png 12 medium_mammals
44018 train_44018.png 11 large_omnivores_and_herbivores
44019 train_44019.png 9 large_man-made_outdoor_things
44020 train_44020.png 12 medium_mammals
44021 train_44021.png 9 large_man-made_outdoor_things
44022 train_44022.png 3 food_containers
44023 train_44023.png 

In [ ]:
import shutil

for file_n in range(len(val_master_trial)):  #testデータを利用
  v_f_name = val_master_trial["file_name"][file_n]
  #v_l_id = val_master_trial["label_id"][file_n]
  #l_name = label_master["label_name"][v_l_id]
  #print(file_n, v_f_name, v_l_id, l_name)
  l_name = "no_label"
  print(file_n, v_f_name, l_name)
  _valdir_path = os.path.join('/','content', 'drive','MyDrive','SIGNATE','Images_labeling', 'train_dir_All','val')
  ret = shutil.copy('/content/drive/MyDrive/SIGNATE/Images_labeling/test/'+v_f_name, _valdir_path)  # dstにディレクトリを指定する
print(ret)  # foo/a.txt：指定したディレクトリの下にファイルをコピー

ストリーミング出力は最後の 5000 行に切り捨てられました。
5001 test_05001.png no_label
5002 test_05002.png no_label
5003 test_05003.png no_label
5004 test_05004.png no_label
5005 test_05005.png no_label
5006 test_05006.png no_label
5007 test_05007.png no_label
5008 test_05008.png no_label
5009 test_05009.png no_label
5010 test_05010.png no_label
5011 test_05011.png no_label
5012 test_05012.png no_label
5013 test_05013.png no_label
5014 test_05014.png no_label
5015 test_05015.png no_label
5016 test_05016.png no_label
5017 test_05017.png no_label
5018 test_05018.png no_label
5019 test_05019.png no_label
5020 test_05020.png no_label
5021 test_05021.png no_label
5022 test_05022.png no_label
5023 test_05023.png no_label
5024 test_05024.png no_label
5025 test_05025.png no_label
5026 test_05026.png no_label
5027 test_05027.png no_label
5028 test_05028.png no_label
5029 test_05029.png no_label
5030 test_05030.png no_label
5031 test_05031.png no_label
5032 test_05032.png no_label
5033 test_05033.png no_label
5034 test_0